## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [ ]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [ ]:
# Create a list of at least 10 column names to use as X data
X_cols = ['Age', 'HourlyRate', 'DistanceFromHome', 'Education', 'YearsAtCompany', 'StockOptionLevel', 'YearsSinceLastPromotion', 'JobInvolvement', 'JobSatisfaction', 'OverTime']


# Create X_df using your selected columns
X_df = attrition_df[X_cols]

# Show the data types for X_df
X_df.dtypes

,0
Age,int64
HourlyRate,int64
DistanceFromHome,int64
Education,int64
YearsAtCompany,int64
StockOptionLevel,int64
YearsSinceLastPromotion,int64
JobInvolvement,int64
JobSatisfaction,int64
OverTime,object


In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)



In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# One-hot encode categorical columns
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
X_train.head()


,Age,HourlyRate,DistanceFromHome,Education,YearsAtCompany,StockOptionLevel,YearsSinceLastPromotion,JobInvolvement,JobSatisfaction,OverTime_No,OverTime_Yes
1097,24,57,21,2,1,3,0,2,1,True,False
727,18,73,5,2,0,0,0,3,4,True,False
254,29,45,20,2,3,1,0,3,4,True,False
1175,39,66,12,3,5,0,1,3,2,True,False
1341,31,89,20,3,10,1,0,3,3,True,False


In [ ]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

# Fit the encoder to the training data
enc.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_enc = enc.transform(y_train[['Department']])
y_test_enc = enc.transform(y_test[['Department']])


In [ ]:
# Create a OneHotEncoder for the Attrition column
enc = OneHotEncoder()

# Fit the encoder to the training data "Attrition" column
# YOUR CODE HERE
enc.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_enc = enc.transform(y_train[['Attrition']])
y_test_enc = enc.transform(y_test[['Attrition']])

type(y_train_enc.toarray())

numpy.ndarray

## Create, Compile, and Train the Model

In [ ]:
# Find the number of columns in the X training data
X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(X_train_scaled.shape[1],))

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(units=32, activation='relu')(shared_layer1)

In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer


# Create the hidden layer
hidden_layer1 = layers.Dense(units=64, activation='relu')(shared_layer2)

# Create the output layer
output_layer1 = layers.Dense(units=2, activation='softmax')(hidden_layer1)

In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
hidden_layer2 = layers.Dense(units=32, activation='relu')(shared_layer2)

# Create the output layer
output_layer2 = layers.Dense(units=2, activation='softmax')(hidden_layer2)


In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs=[output_layer1, output_layer2])


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Summarize the model
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 11)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   768       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 department_output (Dense)   (None, 3)                    99        ['dense_1[0][0]']             
                                                                                            

In [ ]:
# Train the model

# Train the model
# Convert y_train_enc to dense arrays outside of the fit function
y_train_dense_1 = y_train_enc.toarray()
y_train_dense_2 = y_train_enc.toarray() # Create a second array for the second output

model.fit(X_train_scaled, [y_train_dense_1, y_train_dense_2], epochs=100, batch_size=32)


#model.fit(X_train_scaled, [y_train_enc.toarray(), y_train_enc.toarray()], epochs=100, batch_size=32)

Epoch 1/100
37/37 [==============================] - 1s 2ms/step - loss: 1.2407 - dense_2_loss: 0.6359 - dense_6_loss: 0.6048 - dense_2_accuracy: 0.6497 - dense_6_accuracy: 0.6531
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 0.8917 - dense_2_loss: 0.4773 - dense_6_loss: 0.4145 - dense_2_accuracy: 0.8316 - dense_6_accuracy: 0.8359
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 0.8423 - dense_2_loss: 0.4507 - dense_6_loss: 0.3916 - dense_2_accuracy: 0.8316 - dense_6_accuracy: 0.8452
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 0.8179 - dense_2_loss: 0.4368 - dense_6_loss: 0.3811 - dense_2_accuracy: 0.8316 - dense_6_accuracy: 0.8503
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 0.7967 - dense_2_loss: 0.4250 - dense_6_loss: 0.3717 - dense_2_accuracy: 0.8333 - dense_6_accuracy: 0.8520
Epoch 6/100
37/37 [==============================] - 0s 2ms/step - loss: 0.7821 - dense_2_loss: 0.41

In [ ]:
# Evaluate the model with the testing data
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, [y_test_enc.toarray(), y_test_enc.toarray()])

# Extract the loss and metrics
model_loss = results[0]  # Total loss
department_accuracy = results[1]  # Accuracy for the 'Department' branch
attrition_accuracy = results[2]  # Accuracy for the 'Attrition' branch

print("Total Loss:", results)

10/10 [==============================] - 0s 2ms/step - loss: 1.7132 - dense_2_loss: 0.4696 - dense_6_loss: 1.2436 - dense_2_accuracy: 0.8571 - dense_6_accuracy: 0.8639
Total Loss: [1.713224172592163, 0.4695783257484436, 1.2436457872390747, 0.8571428656578064, 0.8639456033706665]


In [ ]:
# Print the accuracy for both department and attrition
print("Department Accuracy:", department_accuracy)
print("Attrition Accuracy:", attrition_accuracy)

Department Accuracy: 0.4695783257484436
Attrition Accuracy: 1.2436457872390747


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE


1. Is accuracy the best metric to use on this data? Why or why not?

- No, accuracy might not be the best metric due to potential class imbalance and the multi-class nature of the Department column. Alternative metrics like F1-score, precision, recall, or ROC-AUC could provide a more comprehensive understanding of the model's performance.
2. What activation functions did you choose for your output layers, and why?


I chose:
- Sigmoid for Attrition (binary classification, predicting probability of leaving)
- Softmax for Department (multi-class classification, predicting probability of each department)


These activation functions are suitable for their respective classification tasks.
3. Can you name a few ways that this model might be improved?


Yes, here are a few ways to improve the model:
- Handle class imbalance using techniques like oversampling, undersampling, or weighted losses
- Experiment with different architectures, such as adding more layers or using transfer learning
- Try alternative activation functions or optimizers
- Use techniques like feature engineering or dimensionality reduction to improve data quality
- Collect more data to increase the model's generalizability